In [2]:
# 导入cv模块
import cv2
#参考
#https://blog.csdn.net/l641208111/article/details/106202138
def insert_pic(img,img2,y_in=0,x_in=0):
    img1=img
    #####################
    # 这里缩小方式用了特殊的技巧
    img2 = cv2.resize(img2,(100,100),cv2.INTER_AREA)

    # I want to put logo on top-left corner, So I create a ROI
    #首先获取原始图像roi
    rows,cols,channels = img2.shape
    roi = img1[x_in:rows+x_in, y_in:cols+y_in ]

    #原始图像转化为灰度值
    # Now create a mask of logo and create its inverse mask also
    img2gray = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)

    #cv2.imshow('img2gray',img2gray)
    #cv2.waitKey(0)
    '''
    将一个灰色的图片，变成要么是白色要么就是黑色。（大于规定thresh值就是设置的最大值（常为255，也就是白色））
    '''
    #将灰度值二值化，得到ROI区域掩模
    ret, mask = cv2.threshold(img2gray, 200, 255, cv2.THRESH_BINARY)

    #cv2.imshow('mask',mask)
    #cv2.waitKey(0)

    #ROI掩模区域反向掩模
    mask_inv = cv2.bitwise_not(mask)

    #cv2.imshow('mask_inv',mask_inv)
    #cv2.waitKey(0)

    #掩模显示背景
    # Now black-out the area of logo in ROI
    img1_bg = cv2.bitwise_and(roi,roi,mask = mask)

    #cv2.imshow('img1_bg',img1_bg)
    #cv2.waitKey(0)

    #掩模显示前景
    # Take only region of logo from logo image.
    img2_fg = cv2.bitwise_and(img2,img2,mask = mask_inv)
    #cv2.imshow('img2_fg',img2_fg)
    #cv2.waitKey(0)

    #前背景图像叠加
    # Put logo in ROI and modify the main image
    dst = cv2.add(img1_bg,img2_fg)
    img1[x_in:rows+x_in, y_in:cols+y_in ] = dst

    #cv2.imshow('res',img1)
    #cv2.waitKey(0)
    #####################
    
    return img1

# insert picture

In [3]:
# 导入cv模块
import cv2

# 导入视频
cap = cv2.VideoCapture(0)

img2 = cv2.imread("hb.jpg")

while cap.isOpened():
    # 读取数据
    success, imgBGR = cap.read()
    if not success:
        print("The vedio is not read successfully")
        continue
     
    img1=insert_pic(imgBGR,img2)
    
    cv2.imshow('res',img1)
    
    # 退出按'q'
    if cv2.waitKey(1) == ord('q'):
        print("Quit successfully")
        break
        
cv2.destroyAllWindows()
# 释放摄像头
cap.release()

Quit successfully


# Picture Moving with head

In [4]:
import cv2
import time
import mediapipe as mp
# 加入时间
import time
import numpy as np


# 导入由于根据坐标画点的函数
mpDraw = mp.solutions.drawing_utils

# 导入模型
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
face_detect = cv2.CascadeClassifier(r"E:\anaconda\Lib\site-packages\opencv\opencv\sources\data\haarcascades\haarcascade_frontalface_default.xml")

# 导入视频
cap = cv2.VideoCapture(0)
# time.sleep(2)

# 更改点和线的设定
handLmsStyle = mpDraw.DrawingSpec(color=(0, 0, 255), thickness=5)
handConStyle = mpDraw.DrawingSpec(color=(0, 255, 0), thickness=10)

faceLmsStyle = mpDraw.DrawingSpec(color=(255, 0, 0), thickness=1)

# 初始化pTime和cTime
pTime = 0
cTime = 0

# 如果有信号则传入
while cap.isOpened():
    # 读取数据
    success, imageBGR = cap.read()
    if not success:
        print("The vedio is not read successfully")
        continue

    imageBGR = cv2.flip(imageBGR, 1)
    # 将其由BGR转为RGB
    image = cv2.cvtColor(imageBGR, cv2.COLOR_BGR2RGB)
    #转为gray类型
    gray = cv2.cvtColor(imageBGR,cv2.COLOR_BGR2GRAY)
    
    #找到画布大小
    imgHeight = imageBGR.shape[0]
    imgWidth = imageBGR.shape[1]
    
    # 找到手部
    # result = hands.process(image)

    # 找到其他部分
    results = holistic.process(image)

    # 导入面部
    #mpDraw.draw_landmarks(imageBGR, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,faceLmsStyle)
    
    # 导入左右手
    mpDraw.draw_landmarks(imageBGR, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, handLmsStyle,
                          handConStyle)
    mpDraw.draw_landmarks(imageBGR, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, handLmsStyle,
                          handConStyle)
    if(results.right_hand_landmarks):
        thumb_right=results.right_hand_landmarks.landmark[4]
        #获得坐标后转成int好看些
        xPos = int(thumb_right.x * imgWidth)
        yPos = int(thumb_right.y * imgHeight)
        cv2.circle(imageBGR,(xPos,yPos),20,(166,56,56),cv2.FILLED)
        
    if(results.left_hand_landmarks):
        thumb_left=results.left_hand_landmarks.landmark[4]
        #获得坐标后转成int好看些
        xPos = int(thumb_left.x * imgWidth)
        yPos = int(thumb_left.y * imgHeight)
        cv2.circle(imageBGR,(xPos,yPos),20,(166,56,56),cv2.FILLED)
        
    # 可以导入动作
#     mpDraw.draw_landmarks(imageBGR, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) 
    
    #插入图片
    #面部识别并且添加图片
    #检测
    face = face_detect.detectMultiScale(gray,1.1,5,0,(150,150),(600,600))
    
    #画图
    for x,y,w,h in face:
        #cv2.rectangle(imageBGR,(x,y),(x+w,y+h),color=(0,0,255),thickness=2)
        #加图
        try:
            insert_pic(imageBGR,img2,x-100,y+50)
        except:
            pass
        try:
            insert_pic(imageBGR,img3,x+w,y+50)
        except:
            pass
    
    # 加入帧数
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    cv2.putText(imageBGR, f"FPS : {int(fps)}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)

    # 展示结果
    cv2.imshow('MediaPipe Holistic', imageBGR)

    # 退出按'q'
    if cv2.waitKey(1) == ord('q'):
        print("Quit successfully")
        break

# 释放内存
cv2.destroyAllWindows()
# 释放摄像头
cap.release()

Quit successfully


# face_location

In [5]:
# 导入库
import cv2 as cv
# 检测函数
def face_detect_demo(img):
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    face_detect = cv.CascadeClassifier(r"E:\anaconda\Lib\site-packages\opencv\opencv\sources\data\haarcascades\haarcascade_frontalface_default.xml")
    face = face_detect.detectMultiScale(gray,1.1,5,0,(150,150),(600,600))
    for x,y,w,h in face:
        cv.rectangle(img,(x,y),(x+w,y+h),color=(0,0,255),thickness=2)
    cv.imshow('result',img)

# 读取摄像头
cap = cv.VideoCapture(0)
# 循环
while True:
    flag,frame = cap.read()
    if not flag:
        break
    face_detect_demo(frame)
    if ord('q') == cv.waitKey(1):
        print("Quit successfully")
        break

# 释放内存
cv.destroyAllWindows()
# 释放摄像头
cap.release()

Quit successfully


# 优化帧率

In [7]:
import cv2
import time
import mediapipe as mp
# 加入时间
import time
import numpy as np


# 导入由于根据坐标画点的函数
mpDraw = mp.solutions.drawing_utils

#导入检测手部的库
mpHands = mp.solutions.hands
hands = mpHands.Hands(model_complexity=0,min_tracking_confidence=0.5)

#导入检测脸部的库
face_detect = cv2.CascadeClassifier(r"E:\anaconda\Lib\site-packages\opencv\opencv\sources\data\haarcascades\haarcascade_frontalface_default.xml")

#导入图片
img2 = cv2.imread("hb.jpg")
img3 = cv2.imread("hp2.jpg")

# 导入视频
cap = cv2.VideoCapture(0)
# time.sleep(2)

# 更改点和线的设定
handLmsStyle = mpDraw.DrawingSpec(color=(0, 0, 255), thickness=5)
handConStyle = mpDraw.DrawingSpec(color=(0, 255, 0), thickness=10)

# 初始化pTime和cTime
pTime = 0
cTime = 0

# 如果有信号则传入
while cap.isOpened():
    # 读取数据
    success, imageBGR = cap.read()
    if not success:
        print("The vedio is not read successfully")
        continue

    imageBGR = cv2.flip(imageBGR, 1)
    # 将其由BGR转为RGB
    image = cv2.cvtColor(imageBGR, cv2.COLOR_BGR2RGB)
    #转为gray类型
    gray = cv2.cvtColor(imageBGR,cv2.COLOR_BGR2GRAY)
    
    # 找到手部
    result = hands.process(image)
        
    if result.multi_hand_landmarks:
        #找到画布大小
        imgHeight = imageBGR.shape[0]
        imgWidth = imageBGR.shape[1]

        #对于找到的每一个手
        for handLms in result.multi_hand_landmarks:
            # 告诉系统话画哪张图上面，并且传入坐标
            # 第一个是画上去的图片，第二个是传入点位坐标，第三个是链接关节点
            #第四个是设定点的样式，第五个是设定线的样式（要在上面添加mpDrwa）
            mpDraw.draw_landmarks(imageBGR,handLms,mpHands.HAND_CONNECTIONS,handLmsStyle,handConStyle)
            #遍历的时候显示是第几个点,找到点的位置
            for i, lm in enumerate(handLms.landmark):
                #获得坐标后转成int好看些
                xPos = int(lm.x * imgWidth)
                yPos = int(lm.y * imgHeight)
                # 找大拇指的位置
                if i == 4: 
                # (图片)，字符串，位置，字体，大小，颜色,粗度
                    cv2.circle(imageBGR,(xPos,yPos),20,(166,56,56),cv2.FILLED)
    
    #面部识别并且添加图片
    #检测
    face = face_detect.detectMultiScale(gray,1.1,5,0,(150,150),(600,600))
    
    #画图
    for x,y,w,h in face:
        #cv2.rectangle(imageBGR,(x,y),(x+w,y+h),color=(0,0,255),thickness=2)
        #加图
        try:
            insert_pic(imageBGR,img2,x-100,y+50)
        except:
            pass
        try:
            insert_pic(imageBGR,img3,x+w,y+50)
        except:
            pass
    
    
    # 加入帧数
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    cv2.putText(imageBGR, f"FPS : {int(fps)}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)

    # 展示结果
    cv2.imshow('imageBGR', imageBGR)

    # 退出按'q'
    if cv2.waitKey(1) == ord('q'):
        print("Quit successfully")
        break

# 释放内存
cv2.destroyAllWindows()
# 释放摄像头
cap.release()

Quit successfully
